In [1]:
from ACDC.random_walk_classifier import * 
from ACDC.cell_type_annotation import * 

In [2]:
import pandas as pd
import numpy as np
from collections import Counter

path = 'data/Samusik01/40000events/'
df = pd.read_csv(path + 'Samusik01_5.csv.gz', sep=',', header = 0, compression = 'gzip')

df = df.drop(['Time', 'Cell_length', 'BC1()', 'BC2()', 'BC3()', 'BC4()', 'BC5()', 'BC6()', 'DNA1()',
              'DNA2()', 'Cisplatin()', 'beadDist()', 'sample', 'event'], axis = 1)

channels = [item[:item.find('(')] for item in df.columns[:-1]]
df.columns = channels + ['cell_type']


#df = df.loc[df['cell_type'] != 'NotDebrisSinglets']

table = pd.read_csv(path + 'Samusik01_table.csv', sep=',', header=0, index_col=0)
table = table.fillna(0)

cts, channels = get_label(table)

#X0= np.arcsinh((df[channels].values - 1.0)/5.0)
X0= df[channels].values

In [3]:
idx2ct = [key for idx, key in enumerate(table.index)]
#idx2ct.append('unknown')

ct2idx = {key:idx for idx, key in enumerate(table.index)}
#ct2idx['unknown'] = len(table.index)
        
ct_score = np.abs(table.as_matrix()).sum(axis = 1)

## compute manual gated label
y0 = np.zeros(df.cell_type.shape)

for i, ct in enumerate(df.cell_type):
    if ct in ct2idx:
        y0[i] = ct2idx[ct]
    #else:
        #y0[i] = ct2idx['unknown']

In [4]:
from sklearn.metrics import accuracy_score, confusion_matrix
import phenograph
from sklearn.cross_validation import StratifiedKFold
import pickle
n_neighbor = 10
thres = 0.5


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
import time
import scipy.io as sio 

skf = StratifiedKFold(y0, n_folds=5, shuffle=True, random_state=0)
result = []
score_final = []


process_time = []
c = 0
for tr, te in skf:
    print('%02d th batch' % c)
    if c == 1:
        break
    c += 1
    
    X = X0.copy()
    y_true = y0.copy()

    X = X[tr, :]
    y_true = y_true[tr]

    mk_model =  compute_marker_model(pd.DataFrame(X, columns = channels), table, 0.0)

    ## compute posterior probs
    tic = time.clock()
    score = get_score_mat(X, [], table, [], mk_model)
    score = np.concatenate([score, 1.0 - score.max(axis = 1)[:, np.newaxis]], axis = 1)    

    ## get indices     
    ct_index = get_unique_index(X, score, table, thres)
    
    ## baseline - classify events    
    y_pred_index = np.argmax(score, axis = 1)
    
    toc = time.clock()
    time0 = toc - tic
    
    
    
    ## running ACDC
    tic = time.clock()
    res_c = get_landmarks(X, score, ct_index, idx2ct, phenograph, thres)

    landmark_mat, landmark_label = output_feature_matrix(res_c, [idx2ct[i] for i in range(len(idx2ct))]) 

    landmark_label = np.array(landmark_label)

    lp, y_pred = rm_classify(X, landmark_mat, landmark_label, n_neighbor)

    process_time.append(toc-tic)
    
    res = phenograph.cluster(X, k=30, directed=False, prune=False, min_cluster_size=10, jaccard=True,
                        primary_metric='euclidean', n_jobs=-1, q_tol=1e-3)
    
    toc = time.clock()
    time1 = toc - tic
    
    
    ## running phenograph classification
    tic = time.clock()
    y_pred_oracle = np.zeros_like(y_true)
    for i in range(max(res[0])+1):
        ic, nc = Counter(y_true[res[0] == i]).most_common(1)[0]
        y_pred_oracle[res[0] == i] = ic
        
    score_final.append([accuracy_score(y_true, [ct2idx[c] for c in y_pred]), 
                    accuracy_score(y_true, y_pred_index), 
                    accuracy_score(y_true, y_pred_oracle)])
    
    toc = time.clock()
    time2 = toc - tic   
    
    
    result.append((y_true, y_pred, y_pred_index, y_pred_oracle))
    process_time.append((time0, time1, time2))
    
    #pickle.dump(result, open('processed_file/AML/event_classidication_AML.p', 'wb'))
    sio.savemat('processed_file/Samusik01/40000events/event_classidication_Samusik01_5.mat',{'y_true':y_true,'y_pred_index':y_pred_index,'y_pred_oracle':y_pred_oracle,'X':X})

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


00 th batch
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 0.2677004337310791 seconds
Jaccard graph constructed in 1.3313302993774414 seconds
Wrote graph to binary file in 0.011000633239746094 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.712212
After 3 runs, maximum modularity is Q = 0.713748
Louvain completed 23 runs in 16.70855474472046 seconds
PhenoGraph complete in 18.32158613204956 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 0.10300588607788086 seconds
Jaccard graph constructed in 1.199068546295166 seconds
Wrote graph to binary file in 0.0070002079010009766 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.53431
After 2 runs, maximum modularity is Q = 0.542497
Louvain completed 22 runs in 2.1281216144561768 seconds
PhenoGraph complete in 3.441196918487549 seconds
Finding 30 nearest neighbors 

In [6]:
np.mean(score_final, axis = 0) # score of ACDC, score-based classification, phenograph classification

array([ 0.95048917,  0.67939862,  0.96874316])

In [7]:
process_time

[-1.9956682595356767e-06,
 (0.5571498093994265, 242.22021229918943, 0.0307672175566438)]